# 4.1 Why scalable computing matters

<img src="./images/scalable_computing.jpg" width="600"/>

## *Subjects covered*

* Dask: a framework for scalable computing
* Directed Acyclic Graphs (DAG): how to read and interpre DAGs
* Why DAGs are useful for distributed workloads
* How Dask scheduler uses DAGs to compose, control and monitor computations
* Introducing the companion dataset

## *Content*

- [Why Dask?](#Why-Dask?)
- [Cooking with DAGs](#Cooking-with-DAGs)
- [Scaling out, concurrency and recovery](#Scaling-out,-concurrency-and-recovery)
- [Companion dataset](#Companion-dataset)
- [Summary](#Summary)

## Why Dask?

Python Open Data Science Stack is a great set of tools for analysis of data

* **Pandas** for data cleaning
* **SciPy** and **Numpy** for scientific computing in general
* **scikit-learn** for predictive modelling
* **Keras**, **TensorFlow**, **PyTorch** for deep learning

... **as long as they fit into RAM of your computer!**

### Rough three-tiered definition of data size

#### Dask for native scalability in Python Open Data Science Stack

Dask consists of several different components
and APIs.

<img src="./images/dask_layers.png" />

### Dask components

**Task scheduler (Dask subsystem)**
* Coordinates and monitors execution of computations across CPU cores and machines

**Dask Delayed objects and Dask Futures objects (low-level APIs)**
* Delayed objects: evaluated *lazily*, i.e. just in time when the values are needed
* Futures object: evaluated *eagerly*, i.e. in real time  regardless if the value is needed immediately or not

**Dask Array, Bag, DataFrame and ML (high-level APIs)**
* Operations on these high level APIs  result in **many prallel low-level** operations
* ==> **Seamless** experience for the user

<img src="./images/Dask_high_level_APIs.png" width="1300"/>

### Key advantages of Dask

* **Fully** implemented in Python and **natively scales** Numpy, Pandas and scikit-learn

* Can be used effectively to work on
    * medium sized datasets on a **single machine**
    * large datasets on a **cluster**

* Can be used as a **general framework** for **parallelising most Python objects**


* Has **very low** configuration and maintenance overhead

### Key features of Dask

* Dask **doesn't merely mirror** common operations and patterns that NumPy and Pandas users will find familiar


* The underlying objects used by Dask **are** corresponding objects from each respective library
    * A Dask DataFrame consists of many Pandas DataFrames
    * A Dask Array consists of many Numpy arrays
    * etc.

* Each of the smaller underlying objects are called **chunks** or **partitions**


* Chunks or partitions **can be shipped** from **machine to machine within a cluster**, or **queued up** and worked on **one piece at a time locally**

### Why not other technologies?

* Well working and well established software like **PySpark** exist
* They typically rely on other languages and technologies, not Python
* Handling **multiple languages / technologies** is often **painfully difficult** 
* ==> **complicates maintenance** and **increases overhead** (need often extra manpower to keep systems up and running)
* Would make parallelism **almost impossible** to achieve to a data scientist


## Cooking with DAGs

* Dask schedulers use concept of **directed acyclic graphs** (DAGs)
* DAGs come from a larger body of mathematics known as *graph theory*
* Graph theory describes a graph as a **representation of a set of objects that have a relationship with one another**
* ==> Graphs are useful for representing a **very wide variety of information**

#### Exemplification of DAGs by following a cooking recipe

<img src="./images/Pasta_recipe.png" />

### Cooking recipe

* Cooking a recipe consists of following a **series of sequential steps**
* raw ingredients **are transformed** into intermediate states
* all the ingredients are ultimately **combined** into a single complete dish

<img src="./images/DAG_01.png" />

### Essential elements of the DAG

* Each *circle* represents a **node** in the graph 
* A node is a **standalone unit of work** and may represent functions
* Each node can take on **dependencies**
    * this means that a **prior** step (or steps) **must be complete** before starting the next node’s operation
* Dependencies are represented by a *connecting line*
* Connecting lines have an *arrow* as endpoints
* ==> This implies that there is **only one possible way** to traverse the graph (hence: directed acyclic graph)

**Transitive dependency**

<img src="./images/DAG_02.png" width="700" />

### The full recipe represented in a DAG

<img src="./images/DAG_03.png" />

* One can start with **any of the blue nodes**, since they have **no** dependencies
* Following the dependencies, one will reach the **terminal node**
* DAGs may be useful to **spot bottlenecks**
* ==> potentially **reorder** some nodes to produce **more optimal** or **more time-efficient** process

## Scaling out, concurrency and recovery

* Previous example assumed only **one cook** worked in kitchen
* One cook **not enough** for hundreds of guests
* Help is needed!

### Scaling up vs. scaling out in computing

**Reasons for scaling up**
* Most of the time, upgrading a **cheap**, **low-end** workstation to a **high-end** server may be a better option than buying lots of new hardware and setting up a cluster
* This is especially true, if size of problem sits at the **high end** of medium datasets or **low end** of large datasets

**Reasons for scaling out**
* If problem to be solved allows for **a lot of parallellism**
* If data dataset is **large**

### Task scheduler in Dask

Dask is a library for taking advantage of parallellism, i.e. scaling out (many cooks in our kitchen example)

The task scheduler
* **divides** and **supervises** the work
* **constantly evaluates** what work needs to be done
* **organises execution of the graph** between workers and **assigning an appropriate number of workers** to each task
* aims to **cycle workers** between many tasks in order to **reduce memory load** and **emit finished results quickly**
* aims to **minimize** the worker pool’s idle time
* **distributes** units of work to **machines** in an efficient manner



### Work distribution in cooking example with 10 workers

<img src="./images/DAG_04.png" />

### Concurrency and resource management

* Often one has to consisder **more constraints** than just the number of workers
* Cooking example: number of knifes **is smaller** than number of workers
* In **scalable computing**, these are called *issues of concurrency*
* Even if the remaining five workers have **completed all other possible nodes**, some steps (nodes) become **delayed** due to *resource starvation*
* Cooking example: The other cooks are **forced** to remain **idle** until the onion-dicing step is complete
* When a **shared resource** is in use, a *resource lock* is placed on it, meaning other workers **can’t** “steal” the resource until the worker who locked the resource **is finished** using it

### Task scheduler in scalable computing frameworks

If not handled properly, resource contention can be **very detrimental** to performance.

* **Decides** how to deal with **resource contention** and **locking**
* **Efficient** task scheduling
* Normally, **no need** to **hand-tune** schedules


### Issues of concurrency in cooking example with idle workers

<img src="./images/resource_starvation.png" />

### Recovery from failures

* It gets **increasingly difficult** to **orchestrate distribution of processing tasks** as the number of machines in a cluster **increases**
* The final result consists of the **aggregate of all the individual operations**
* ==> Important to ensure that **all the pieces** find their way to where they need to go
* But machines, like people, are **imperfect** and **fail** at times
* Two types of failures:
    * worker failure **without** data loss
    * worker failure **with** data loss

**Worker failure**

**Worker failure without data loss** 
* A worker fails 
* Another worker takes over without need to reproduce failed worker's results 
* ==> Less severe impact on performance


**Worker failure with data loss** 
* The dependencies for the particular node are no longer met
* Worker need to step all the way back to the first dependency-free node and work your way back from there
* ==> Severe impact on performance

**Worker failure**

* At **any point** in the graph, the **complete lineage of operations** up to a given node can be “replayed” in the event of a failure
* The task scheduler is ultimately responsible for **stopping work** and **redistributing the work** to be replayed
* Because the task scheduler can **dynamically redistribute tasks** away from failed workers, the specific workers that completed the tasks before **don’t** need to be present to **redo** the tasks

## Companion dataset

* Apply your newfound skills to a **real**, **messy** dataset
* Gain experience using an **appropriately large** dataset
* Data are fetched from [NYC OpenData](https://opendata.cityofnewyork.us)
    * [NYC Parking Tickets on Kaggle](https://www.kaggle.com/new-york-city/nyc-parking-tickets)
    * Every third week of the month, the New York City Department of Finance records and publishes a data set of all parking citations issued throughout the fiscal year
    *  The data are an archive containing four years ofdata from NYC OpenData has been collected and published on the popular machine learning website, Kaggle
    * Spans from 2013 through June 2017 and is over 8 GB uncompressed

<img src="./images/NYC_open_data.png" width=500/>

## Summary

* Dask can be used to **scale** popular data analysis libraries such as Pandas and NumPy, allowing you to analyze medium and large datasets with ease.
* Dask uses **directed acyclic graphs (DAGs)** to coordinate execution of **parallelized code** across **CPU cores** and **machines**.
* Directed acyclic graphs are made up of **nodes** and have a **clearly defined start** and **end**, a **single traversal path**, and **no looping**.
* **Upstream nodes** must be completed **before** work can begin on any **dependent downstream nodes**.
* **Scaling out** can generally **improve** performance of **complex workloads**, but it creates **additional overhead** that might substantially **reduce** those performance gains.
* In the event of a **failure**, the steps to reach a node can be **repeated from the beginning without disturbing the rest of the process**.